In [21]:
def _generate_trial_sxx_data(data_func, indices):
    sxx_data_dict = data_func()
    
    sxx = sxx_data_dict['sxx']
    
    return sxx[:, indices, :].astype(np.float16)

# def _window_spectrogram(data_func, indices, shift, window_size):
#     sxx_data_dict = data_func()
    
#     sxx = sxx_data_dict['sxx']
    
#     return np.moveaxis(sliding_window_view(sxx[:, indices, :][:, ::shift, :], window_shape=window_size, axis=1), [0, -1], [1, -2])
    
def _window_spectrogram(sxx, shift, window_size):
    return np.moveaxis(sliding_window_view(sxx, window_shape=window_size, axis=1), [0, -1], [1, -2])[::shift, ...]

def generate_model_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id, spectrogram_params):
    pre_stimulus_time = model_data_params['pre_stimulus_time']
    post_completion_time = model_data_params['post_completion_time']

    window_size = model_data_params['window_size']
    shift = spectrogram_params['shift']

    model_data_dict = {}
    model_data_filenames_dict = {}
    global_trial_idx = 0
    model_data_filenames = []
    for session_type, session_data in sessions[patient_id][current_experiment].items():
        for sxx_partition_key, sxx_partition_func in spectrogram_dict.items():
            date = sxx_partition_key.split('_')[-2]
            session = sxx_partition_key.split('_')[-1]

            if date in session_data.keys() and session in session_data[date]:
                curated_states_data_func = curated_states_partition[sxx_partition_key]
                curated_states_dict = curated_states_data_func()
                # sxx_data_dict = sxx_partition_func()

                # sampling_rate = sxx_data_dict['sampling_rate']
#                 sampling_rate = 1000

                model_data_metadata = {}
                for state, state_information in curated_states_dict.items():
                    cur_dict = {}

                    start_end_idx = state_information['start_end_idx']
                    
                    sampling_rate = state_information['sampling_rate']
                    
                    samples_pre = int(np.ceil((pre_stimulus_time * sampling_rate)/shift))
                    samples_post = int(np.ceil((post_completion_time * sampling_rate)/shift)) 
                    
                    # print(samples_pre)
                    # print(samples_post)
                    
                    cur_dict['start_end_idx'] = [(x[0]-samples_pre, x[1]+samples_post) for x in start_end_idx]
                    cur_dict['unique_val_idx'] = [np.arange(x[0], x[1]+1) for x in cur_dict['start_end_idx']]
                    cur_dict['num_steps'] = [len(x) for x in cur_dict['unique_val_idx']]

                    model_data_metadata[state] = cur_dict

                # sxx = sxx_data_dict['sxx']

                intermed_model_dict = {}
                for state, metadata_dict in model_data_metadata.items():
                    if state == 0:
                        continue
                        
                    trial_idx_list = []
                    local_trial_idx_list = []
                    for trial_idx, indices in enumerate(metadata_dict['unique_val_idx']):
                        model_data_dict[f"{session_type}_{date}_{session}_T{trial_idx}_state{state}"] = create_closure_func(_generate_trial_sxx_data, sxx_partition_func, indices)
                        model_data_filenames.append(f"{session_type}_{date}_{session}_T{trial_idx}_state{state}")

                        trial_idx_list.append(global_trial_idx)
                        global_trial_idx += 1
                        
                        local_trial_idx_list.append(trial_idx)
                        
                    cur_partition_dict = model_data_filenames_dict.setdefault(sxx_partition_key, {})

                    cur_partition_dict[state] = {
                            'date': date,
                            'session_type': session_type,
                            'session': session,
                            'state': state,
                            'local_trials_idx_list': local_trial_idx_list,
                            'global_trials_idx_list': trial_idx_list
                    }
                        
    return model_data_dict, model_data_filenames_dict, model_data_filenames

def _generate_train_test_split_indices(model_data_filenames_dict, model_data_filenames, model_data_params, current_experiment):
    data_split_type = model_data_params[current_experiment]['sel_split_type']

    split_type_params = model_data_params[current_experiment]['split_types'][data_split_type]

    leave_out = split_type_params['leave_out']
    randomized = split_type_params['randomized']
    random_seed = split_type_params['random_seed']
    sel_session_type = model_data_params[current_experiment]['sel_session_type']

    np.random.seed(random_seed)

    if data_split_type == 'leave_day_out':
        dates_list = []
        for partition_key, partition_data_func in model_data_filenames_dict.items():
            date = partition_key.split('_')[-2]
            session = partition_key.split('_')[-1]
            
            data_dict = partition_data_func 
            
            for state, trial_information_dict in data_dict.items():
                if sel_session_type != trial_information_dict['session_type']:
                    break
                
                if date not in dates_list:
                    dates_list.append(date)
                    break
        
        dates_list = np.array(list(set(dates_list)))

        dates_list_permuted = dates_list
        if randomized:
            dates_list_permuted = np.random.permutation(dates_list)

        test_list_dates = dates_list_permuted[:leave_out]
        train_list_dates = dates_list_permuted[leave_out:]
        
        train_list = []
        test_list = []
        train_labels_list = []
        test_labels_list = []
        for partition_key, partition_data_func in model_data_filenames_dict.items():
            date = partition_key.split('_')[-2]
            session = partition_key.split('_')[-1]
            
            data_dict = partition_data_func()
            
            for state, trial_information_dict in data_dict.items():
                session_type = trial_information_dict['session_type']
                
                if sel_session_type != session_type:
                    continue
                
                if date in train_list_dates:
                    train_list += trial_information_dict['global_trials_idx_list']
                    train_labels_list += [state]*len(trial_information_dict['global_trials_idx_list'])
                    
                elif date in test_list_dates:
                    test_list += trial_information_dict['global_trials_idx_list']
                    test_labels_list += [state]*len(trial_information_dict['global_trials_idx_list'])

    elif data_split_type == 'leave_session_out':
        dates_and_sessions_list = []
        for partition_key, partition_data_func in model_data_filenames_dict.items():
            date = partition_key.split('_')[-2]
            session = partition_key.split('_')[-1]
            
            data_dict = partition_data_func()
            
            for state, trial_information_dict in data_dict.items():
                if sel_session_type != trial_information_dict['session_type']:
                    break
                
                if {'date': date, 'session': session} not in dates_and_sessions_list:
                    dates_and_sessions_list.append({'date': date, 'session': session})
                    break

        dates_and_sessions_list = np.array(dates_and_sessions_list)

        dates_and_sessions_list_permuted = dates_and_sessions_list
        if randomized:
            dates_and_sessions_list_permuted = np.random.permutation(dates_and_sessions_list)
        
        test_list_dates_and_sessions = dates_and_sessions_list_permuted[:leave_out]
        train_list_dates_and_sessions = dates_and_sessions_list_permuted[leave_out:]
        
        train_list = []
        test_list = []
        train_labels_list = []
        test_labels_list = []
        for partition_key, partition_data_func in model_data_filenames_dict.items():
            date = partition_key.split('_')[-2]
            session = partition_key.split('_')[-1]
            
            data_dict = partition_data_func ()
            for state, trial_information_dict in data_dict.items():
                if {'date': date, 'session': session} in train_list_dates_and_sessions:
                    train_list += trial_information_dict['global_trials_idx_list']
                    train_labels_list += [state]*len(trial_information_dict['global_trials_idx_list'])
                    
                elif {'date': date, 'session': session} in test_list_dates_and_sessions:
                    test_list += trial_information_dict['global_trials_idx_list']
                    test_labels_list += [state]*len(trial_information_dict['global_trials_idx_list'])

    elif data_split_type == 'leave_trial_out':
        trials_list = []
        labels_list= []
        for partition_key, partition_data_func in model_data_filenames_dict.items():
            date = partition_key.split('_')[-2]
            session = partition_key.split('_')[-1]
            
            data_dict = partition_data_func()
            
            for state, trial_information_dict in data_dict.items():
                if sel_session_type != trial_information_dict['session_type']:
                    break
                    
                trials_list += trial_information_dict['global_trials_idx_list']
                labels_list += [state]*len(trial_information_dict['global_trials_idx_list'])

        trials_list = np.array(trials_list)
        labels_list = np.array(labels_list)

        permutation_indices = np.arange(len(labels_list))
        if randomized:
            permutation_indices = np.random.permutation(permutation_indices)

        trials_list_permuted = trials_list[permutation_indices]
        trials_labels_list_permuted = labels_list[permutation_indices]
        
        test_list = trials_list_permuted[:leave_out]
        train_list = trials_list_permuted[leave_out:]

        test_labels_list = trials_labels_list_permuted[:leave_out]
        train_labels_list = trials_labels_list_permuted[leave_out:]

    
    return {
        'train_list_idx': np.array(train_list),
        'test_list_idx': np.array(test_list),
        'train_labels_list': np.array(train_labels_list),
        'test_labels_list': np.array(test_labels_list),
        'train_filenames': model_data_filenames[np.array(train_list)],
        'test_filenames': model_data_filenames[np.array(test_list)]
    }

In [19]:
curated_states_partition = catalog.load("center_out_curated_states_pkl")
spectrogram_dict = catalog.load("center_out_spectrogram_std_pkl")
center_out_model_filenames = catalog.load("center_out_model_filenames")

[12/06/22 16:02:56] INFO     Loading data from 'center_out_curated_states_pkl'                  ]8;id=823437;file:///home/ssah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=751969;file:///home/ssah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from 'center_out_spectrogram_std_pkl'                 ]8;id=764113;file:///home/ssah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=298750;file:///home/ssah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from 'center_out_model_filenames' (PickleDataSet)...  ]8;id=297899;file:///home/ssah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=148756;file:///home/ssah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [22]:
model_data_params = context.params['model_data_params']

pre_stimulus_time = model_data_params['pre_stimulus_time']
post_completion_time = model_data_params['post_completion_time']

window_size = model_data_params['window_size']
shift = model_data_params['shift']

current_experiment = context.params['current_experiment']
sessions = context.params['sessions']
patient_id = context.params['patient_id']

spectrogram_params = context.params['spectrogram_params']

model_data_dict, model_data_filenames_dict, model_data_filenames = generate_model_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id, spectrogram_params)

In [23]:
_generate_train_test_split_indices(model_data_filenames_dict, model_data_filenames, model_data_params, current_experiment)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1057610/918837696.py:1 in <cell line: 1>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1057610/918837696.py'                       │
│                                                                                                  │
│ /tmp/ipykernel_1057610/2768308025.py:111 in _generate_train_test_split_indices                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1057610/2768308025.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'dict' object is not callable

In [8]:
def _generate_trial_sxx_data(data_func, indices):
    sxx_data_dict = data_func()
    
    sxx = sxx_data_dict['sxx']
    
    return sxx[:, indices, :].astype(np.float16)
    
def _window_spectrogram(sxx, shift, window_size):
    return np.moveaxis(sliding_window_view(sxx, window_shape=window_size, axis=1), [0, -1], [1, -2])[::shift, ...]

def generate_model_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id):
    pre_stimulus_time = model_data_params['pre_stimulus_time']
    post_completion_time = model_data_params['post_completion_time']

    window_size = model_data_params['window_size']
    shift = model_data_params['shift']

    model_data_dict = {}
    model_data_filenames_dict = {}
    global_trial_idx = 0
    model_data_filenames = []
    for session_type, session_data in sessions[patient_id][current_experiment].items():
        for sxx_partition_key, sxx_partition_func in spectrogram_dict.items():
            date = sxx_partition_key.split('_')[-2]
            session = sxx_partition_key.split('_')[-1]

            if date in session_data.keys() and session in session_data[date]:
                curated_states_data_func = curated_states_partition[sxx_partition_key]
                curated_states_dict = curated_states_data_func()

                model_data_metadata = {}
                for state, state_information in curated_states_dict.items():
                    cur_dict = {}

                    start_end_idx = state_information['start_end_idx']
                    
                    sampling_rate = state_information['sampling_rate']
                    
                    samples_pre = int(np.ceil((pre_stimulus_time * sampling_rate)/shift))
                    samples_post = int(np.ceil((post_completion_time * sampling_rate)/shift)) 
                    
                    print(samples_pre)
                    print(samples_post)
                    
                    cur_dict['start_end_idx'] = [(x[0]-samples_pre, x[1]+samples_post) for x in start_end_idx]
                    cur_dict['unique_val_idx'] = [np.arange(x[0], x[1]+1) for x in cur_dict['start_end_idx']]
                    cur_dict['num_steps'] = [len(x) for x in cur_dict['unique_val_idx']]

                    model_data_metadata[state] = cur_dict
                    
                intermed_model_dict = {}
                for state, metadata_dict in model_data_metadata.items():
                    if state == 0:
                        continue
                        
                    trial_idx_list = []
                    local_trial_idx_list = []
                    for trial_idx, indices in enumerate(metadata_dict['unique_val_idx']):
                        model_data_dict[f"{session_type}_{date}_{session}_T{trial_idx}_state{state}"] = create_closure_func(_generate_trial_sxx_data, sxx_partition_func, indices)
                        model_data_filenames.append(f"{session_type}_{date}_{session}_T{trial_idx}_state{state}")

                        trial_idx_list.append(global_trial_idx)
                        global_trial_idx += 1
                        
                        local_trial_idx_list.append(trial_idx)
                        
                    cur_partition_dict = model_data_filenames_dict.setdefault(sxx_partition_key, {})

                    cur_partition_dict[state] = {
                            'date': date,
                            'session_type': session_type,
                            'session': session,
                            'state': state,
                            'local_trials_idx_list': local_trial_idx_list,
                            'global_trials_idx_list': trial_idx_list
                    }
                        
    return model_data_dict, model_data_filenames_dict, model_data_filenames

In [9]:
generate_model_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id)

500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500
0
500


(
    {
        'overt_20221027_S03_T0_state1': <function create_closure_func.<locals>.closure at 0x7f3f4cb48ca0>,
        'overt_20221027_S03_T1_state1': <function create_closure_func.<locals>.closure at 0x7f3f4e3bac10>,
        'overt_20221027_S03_T2_state1': <function create_closure_func.<locals>.closure at 0x7f3f4d9a55e0>,
        'overt_20221027_S03_T3_state1': <function create_closure_func.<locals>.closure at 0x7f3f4e3cddc0>,
        'overt_20221027_S03_T4_state1': <function create_closure_func.<locals>.closure at 0x7f3f4d84d5e0>,
        'overt_20221027_S03_T0_state2': <function create_closure_func.<locals>.closure at 0x7f3f4d84d940>,
        'overt_20221027_S03_T1_state2': <function create_closure_func.<locals>.closure at 0x7f3f4d84df70>,
        'overt_20221027_S03_T2_state2': <function create_closure_func.<locals>.closure at 0x7f3f4d84d550>,
        'overt_20221027_S03_T3_state2': <function create_closure_func.<locals>.closure at 0x7f3f4d84d670>,
        'overt_20221027_S03_T4_state2': <function create_closure_func.<locals>.closure at 0x7f3f4d849040>,
        'overt_20221027_S03_T0_state3': <function create_closure_func.<locals>.closure at 0x7f3f4d8490d0>,
        'overt_20221027_S03_T1_state3': <function create_closure_func.<locals>.closure at 0x7f3f4d849160>,
        'overt_20221027_S03_T2_state3': <function create_closure_func.<locals>.closure at 0x7f3f4d8491f0>,
        'overt_20221027_S03_T3_state3': <function create_closure_func.<locals>.closure at 0x7f3f4d849280>,
        'overt_20221027_S03_T4_state3': <function create_closure_func.<locals>.closure at 0x7f3f4d849310>,
        'overt_20221027_S03_T0_state4': <function create_closure_func.<locals>.closure at 0x7f3f4d8493a0>,
        'overt_20221027_S03_T1_state4': <function create_closure_func.<locals>.closure at 0x7f3f4d849430>,
        'overt_20221027_S03_T2_state4': <function create_closure_func.<locals>.closure at 0x7f3f4d8494c0>,
        'overt_20221027_S03_T3_state4': <function create_closure_func.<locals>.closure at 0x7f3f4d849550>,
        'overt_20221027_S03_T4_state4': <function create_closure_func.<locals>.closure at 0x7f3f4d8495e0>,
        'overt_20221027_S03_T0_state5': <function create_closure_func.<locals>.closure at 0x7f3f4d849670>,
        'overt_20221027_S03_T1_state5': <function create_closure_func.<locals>.closure at 0x7f3f4d849700>,
        'overt_20221027_S03_T2_state5': <function create_closure_func.<locals>.closure at 0x7f3f4d849790>,
        'overt_20221027_S03_T3_state5': <function create_closure_func.<locals>.closure at 0x7f3f4d849820>,
        'overt_20221027_S03_T4_state5': <function create_closure_func.<locals>.closure at 0x7f3f4d8498b0>,
        'overt_20221027_S03_T0_state6': <function create_closure_func.<locals>.closure at 0x7f3f4d849940>,
        'overt_20221027_S03_T1_state6': <function create_closure_func.<locals>.closure at 0x7f3f4d8499d0>,
        'overt_20221027_S03_T2_state6': <function create_closure_func.<locals>.closure at 0x7f3f4d849a60>,
        'overt_20221027_S03_T3_state6': <function create_closure_func.<locals>.closure at 0x7f3f4d849af0>,
        'overt_20221027_S03_T4_state6': <function create_closure_func.<locals>.closure at 0x7f3f4d849b80>,
        'overt_20221027_S03_T0_state7': <function create_closure_func.<locals>.closure at 0x7f3f4d849c10>,
        'overt_20221027_S03_T1_state7': <function create_closure_func.<locals>.closure at 0x7f3f4d849ca0>,
        'overt_20221027_S03_T2_state7': <function create_closure_func.<locals>.closure at 0x7f3f4d849d30>,
        'overt_20221027_S03_T3_state7': <function create_closure_func.<locals>.closure at 0x7f3f4d849dc0>,
        'overt_20221027_S03_T4_state7': <function create_closure_func.<locals>.closure at 0x7f3f4d849e50>,
        'overt_20221027_S03_T0_state8': <function create_closure_func.<locals>.closure at 0x7f3f4d849ee0>,
        'overt_20221027_S03_T1_state8': <function create_closure_func.<locals>.closure at 0x7f3f4d849f70>,
        'overt_20221027_S03_T2_st

In [6]:
curated_states_dict = curated_states_partition['ccCenterOut_20221027_S03']()
sxx_data_dict = spectrogram_dict['ccCenterOut_20221027_S03']()

In [7]:
curated_states_dict

{
    0: {
        'start_end_idx': [
            (149, 175),
            (243, 263),
            (331, 355),
            (435, 457),
            (519, 543),
            (609, 643),
            (712, 731),
            (807, 835),
            (1019, 1041),
            (1109, 1129),
            (1207, 1223),
            (1287, 1309),
            (1379, 1411),
            (1477, 1513),
            (1585, 1603),
            (1665, 1685),
            (1863, 1897),
            (1971, 2001),
            (2083, 2109),
            (2187, 2203),
            (2267, 2297),
            (2359, 2385),
            (2455, 2487),
            (2561, 2595),
            (2767, 2785),
            (2857, 2881),
            (2945, 2961),
            (3023, 3043),
            (3117, 3143),
            (3199, 3219),
            (3289, 3309),
            (3369, 3391),
            (3561, 3583),
            (3647, 3665),
            (3737, 3751),
            (3835, 3845),
            (3909, 3931),
            (4007, 4025),
            (4121, 4137)
        ],
        'unique_val_idx': [
            array([149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175]),
            array([243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255,
       256, 257, 258, 259, 260, 261, 262, 263]),
            array([331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343,
       344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355]),
            array([435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447,
       448, 449, 450, 451, 452, 453, 454, 455, 456, 457]),
            array([519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531,
       532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543]),
            array([609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621,
       622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634,
       635, 636, 637, 638, 639, 640, 641, 642, 643]),
            array([712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724,
       725, 726, 727, 728, 729, 730, 731]),
            array([807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819,
       820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832,
       833, 834, 835]),
            array([1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029,
       1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040,
       1041]),
            array([1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119,
       1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129]),
            array([1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217,
       1218, 1219, 1220, 1221, 1222, 1223]),
            array([1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297,
       1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308,
       1309]),
            array([1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389,
       1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400,
       1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411]),
            array([1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487,
       1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498,
       1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509,
       1510, 1511, 1512, 1513]),
            array([1585, 1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595,
       1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603]),
            array([1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675,
       1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685]),
            array([1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873,
       1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884,
       1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895,
  